# Q Learning 

In [1]:
# Find the value function of policy
import numpy as np

# display output
from random import uniform
import random
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.pos_check = [0, 0] # a copy of new position
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):
        # return initial state
        return grid.states[gridSize*gridSize-1]
       
    def reward(self, current_pos, action):
        # return the reward        
        
        # take action in current pos
        self.new_pos = np.array(current_pos) + np.array(action)

        # normally, reward = 0
        reward = 0

        # if new pos results in off the grid, return reward -1
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            reward = 5
        return reward
    
    def p_transition(self, current_pos, action):
        # return the transition probability
        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)
        self.pos_check = self.new_pos # make a copy of new pos before being overwritten below

        # if taking an action crosses the border = agent stays in same position
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]
        return self.new_pos

In [4]:
# create a grid object
grid = Gridworld(5)

In [5]:
# get initial state (bottom right)
grid.initial_state()

[4, 4]

## Q Learning

In [6]:
# initialize q values for all state action pairs
Q_values = np.zeros((state_count, action_count))

# initialize other parameters
epsilon = 0.2
# state = 1

In [48]:
def choose_action(state, epsilon):
    
    # choose an action type: explore or exploit
    action_type = int(np.random.choice(2, 1, p=[epsilon,1-epsilon]))

    # find best action based on Q values
    best_action = np.argmax(Q_values[state])

    # pick a random action
    random_action = random.choice(range(4))

    # while random action is the same as the best action, pick a new action
    while random_action == best_action:
        random_action = random.choice(range(4))

    # choose an action based on exploit or explore
    if action_type == 0:
        # explore
        action = random_action
    else:
        # explore
        action = best_action
    
    return action

In [140]:
state = grid.initial_state()

In [143]:
state_index = grid.states.index(state)

In [177]:
chosen_action = choose_action(state_index, epsilon)
chosen_action

2

In [166]:
Q_values[state_index]

array([7, 2, 9, 4])

In [152]:
next_state = grid.p_transition(state, actions[2])

In [153]:
next_state

[4, 4]

In [154]:
chosen_action

4

In [155]:
Q_values[state_index][chosen_action]

IndexError: index 4 is out of bounds for axis 0 with size 4

# Testing 

In [77]:
# # randomly generate Q values (for testing purpose)
# Q_values = np.random.randint(10, size=(state_count, action_count))

In [136]:
# epsilon = 0.2

# state = 1

# # choose an action type: explore or exploit
# action_type = int(np.random.choice(2, 1, p=[epsilon,1-epsilon]))

# print("action type: ", action_type)

# # find best action based on Q values
# best_action = np.argmax(Q_values[state])

# print("best action: ", best_action)

# # pick a random action
# random_action = random.choice(range(4))

# # while random action is the same as the best action, pick a new action
# while random_action == best_action:
#     random_action = random.choice(range(4))
    
# if action_type == 0:
#     # explore
#     action = random_action
# else:
#     # explore
#     action = best_action
    
# print("Action: ", action)

action type:  0
best action:  1
Action:  3
